In [ ]:
import numpy as np 
import pandas as pd 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

#Importation de la librairie FastAI
from fastai import *
from fastai.vision import *
from torchvision.models import *

#Utilisation de la séparation train/test de sklearn
from sklearn.model_selection import train_test_split  

print("Affichage de notre environnement de travail")
print(os.listdir("../input"))

data = pd.read_csv('/kaggle/input/train_labels.csv')
#Configuration du chemin d'entrainement
train_path = '/kaggle/input/train/'
#Configuration du chemin de test
test_path = '/kaggle/input/test/'

# Préparation des données et séparation du set d'entrainement

Séparation des données avec les proportions suivantes : 90% pour l'entrainement et 10% pour la validation.

In [ ]:
train_df = data.set_index('id')
train_names = train_df.index.values
train_labels = np.asarray(train_df['label'].values)
tr_n, tr_idx, val_n, val_idx = train_test_split(train_names, range(len(train_names)), test_size=0.1, stratify=train_labels, random_state=123)

On créé ensuite un panda dataframe pour l'entrainement et un panda dataframe pour le test

In [ ]:
# Création du dataframe d'entrainement
train_dict = {'name': train_path + train_names, 'label': train_labels}
df = pd.DataFrame(data=train_dict)
# Création du dataset de test
test_names = []
for f in os.listdir(test_path):
    test_names.append(test_path + f)
df_test = pd.DataFrame(np.asarray(test_names), columns=['name'])

# Chargement et visualisation des données

FastAI dispose d'objets spécifiques appelés "DataBunch" alimentant les modèles. Ces données nous permettent de charger notre ensemble d'entrainement à partir d'un DataFrame, de spécifier l'ensemble de test, nous permets de spécifier des transformations, et d'appliquer une normalisation

In [ ]:
#On créé ici notre dataBunch, équivalent d'un flow de données alimentant notre modèle de deep learning
tfms = get_transforms()
BATCH_SIZE=128
SIZE=90
data = ImageList.from_df(path='/', df=df, suffix='.tif').split_by_idx(val_idx).label_from_df(cols='label').add_test(ImageList.from_df(path='/', df=df_test)).databunch(bs=BATCH_SIZE).normalize(imagenet_stats)

Maintenant que notre DataBunch a été généré, nous pouvons obtenir d'avantages d'informations sur nos données. Commençons par une visualisation : 

In [ ]:
#Affichage de données sans et avec cancer
data.show_batch(rows=3, figsize=(7, 8))

Regardons de plus près notre objet ImageDataBunch. Nous pouvons récupérer les informations suivantes : 
* Notre ensemble d'entrainement est composé de 198022 éléments
* Notre ensemble de validation est composé de 22003 éléments
* Notre ensemble de test est composé de 57458 éléments
* Chaque image est de dimension 3 (R,G,B) et de taille 96 par 96.
* Notre ensemble d'entrainement possède une image et son label, mais par notre ensemble de validation

In [ ]:
print(data)

# Création du modèle et entrainement initial

Nous utiliserons un modèle de deeplearning, pré entrainé, appelé densenet169 préformé et nous appliquerons la technique du "transfert learning" pour ajuster les paramètres à nos données. L'adoption d'une architecture deep learning permettra de converger plus rapidement et d'obtenir une solution de meilleure qualité.


In [ ]:
# L'architecture de notre modèle, ici densenet169 
arch = densenet169    
# Exraction du nom du modèle
MODEL_PATH = str(arch).split()[1] 
#Création de notre learner, réseau convolutif d'architecture densenet169
learn = create_cnn(data, arch, metrics=error_rate)

Nous utiliserons la méthode du "One cycle policy" proposée par Leslie Smith,[arXiv, avril 2018] (https://arxiv.org/abs/1803.09820). Il s'agit d'appliqué une certaine discipline dans la sélection des hyperparamètres tels que le taux d'apprentissage. 
Afin d'appliquer cette approche, la bibliothèque FastAI a mis en œuvre une fonction, "fit_one_cycle".

In [ ]:
# Utilisation de la GPU
defaults.device = torch.device('cuda') 
# Premier entrainement du modèle
learn.fit_one_cycle(4)
#Enregistrement du modèle
learn.save('cancer-detection-1')

# Amélioration de notre modèle initial
FastAI fournit une autre technique pour améliorer l'apprentissage par transfert learning appelé "differential learning rate", qui nous permet de définir différents taux d'apprentissage pour les différentes couches du réseau.
Pour trouver le meilleur taux d'apprentissage, nous pouvons utiliser les méthodes lr_find et recorder.plot qui créent une courbe reliant le taux d'apprentissage à la perte.

In [ ]:
# On dégèle les dernière couches du modèles pour l'entrainer une seconde fois par la suite
learn.unfreeze() 
#Avant de réentrainer le modèle, on recherche du meilleur taux d'apprentissage en suivant la technique "différential learning late".
learn.lr_find()
learn.recorder.plot()

Nous recherchons ici le point le plus bas, ici 1e-4
Pour former maintenant le modèle en utilisant des taux d'apprentissage différentiels, nous devons passer l'argument max_lr à la méthode fit_one_cycle.

In [ ]:
#Via l'analyse graphique, le meilleur taux d'apprentissage se trouver être 4e-4
learn.fit_one_cycle(4, max_lr=1e-4)

In [ ]:
#Enregistrement du modèle
learn.save('cancer-detection-2')

# Vérification de notre modèle

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

La matrice de confusion peut nous aider à comprendre le taux de faux négatifs et de faux positifs afin de nous permettre d'éxaminer les performances de notre modèle. Ici, nous pouvons voir que le modèle a appris à distinguer la tumeur et l'échantillon négatif et qu'il fonctionne comme nous le désirons.

In [ ]:
learn.recorder.plot_lr()
learn.recorder.plot_losses()

Nous pouvons voir que le taux d'apprentissage commence au plus bas et atteint finalement le max_lr (4e-4) au milieu. Puis il ralentit vers la fin. L'idée est de commencer avec un faible taux d'apprentissage pour l'augmenter jusqu'à un niveau élevé. Le taux plus élevé a un effet de régularisation et évite de tomber dans des minimums locaux en forçant le modèle à apprendre de la manière la plus large possible

# Validation du modèle

En prenant en compte le set de validation, la précision est la suivante:

In [ ]:
preds,y, loss = learn.get_preds(with_loss=True)
acc = accuracy(preds, y)
print('The accuracy is {0} %.'.format(acc))

Affichage de l'air sous la courbe

In [ ]:
from sklearn.metrics import roc_curve, auc
probs = np.exp(preds[:,1])
fpr, tpr, thresholds = roc_curve(y, probs, pos_label=1)

roc_auc = auc(fpr, tpr)
print('ROC area is {0}'.format(roc_auc))
plt.figure()
plt.plot(fpr, tpr, color='darkorange', label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.01])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

# Soumission des résultats

Nous soumettons nos prédictions en appliquant une inférence sur l'ensemble de nos données de test. Le résultat est une probabilité entre 0 et 1 d'avoir évalué une cellule cancereuse.

In [ ]:
#Inférence sur notre test_set
preds,y = learn.get_preds(ds_type=DatasetType.Test, with_loss=False)
tumor_preds = preds[:, 1]

In [ ]:
#Exporter sous le bon format pour la soumission
SAMPLE_SUB = '/kaggle/input/sample_submission.csv'
sample_df = pd.read_csv(SAMPLE_SUB)
sample_list = list(sample_df.id)
pred_list = [float(p) for p in tumor_preds]
pred_dic = dict((key, value) for (key, value) in zip(learn.data.test_ds.items, pred_list))
pred_list_cor = [pred_dic['///kaggle/input/test/' + id + '.tif'] for id in sample_list]
df_sub = pd.DataFrame({'id':sample_list,'label':pred_list_cor})
df_sub.to_csv('{0}_submission.csv'.format(MODEL_PATH), header=True, index=False)
df_sub